# Memória, Variáveis, Comunicação e Estado de um Programa

Há várias possíveis estratégias para usar a memória de um computador para se realizar a comunicação entre partes dele -- por exemplo, entre duas funções -- e para se manter o **estado atual** de execução de um programa. Dentre elas, podemos destacar o uso de variáveis locais, globais e parâmetros entre funções.

Vamos fazer aqui um exercício nesse contexto e você será conduzido a experimentar cada uma dessas estratégias, mesmo que em algumas ocasiões não seja a forma recomendada de se desenvolver um programa.


# Exercício do Empréstimo

Em um financiamento com juros compostos e número de parcelas fixas parte-se dos seguintes parâmetros:
* `S` - valor da primeira parcela
* `N` - número de parcelas
* `J` - percentual de juros mensal

A primeira parcela a ser paga do financiamento é sempre igual a `S`. A partir daí é feita uma atualização mensal da parcela, em que cada nova parcela é calculada a partir da parcela do mês anterior, conforme a fórmula:

> Parcela<sub>mês</sub> = Parcela<sub>mês-1</sub> * (1 + `J` / 100)

O financiamento encerra quando as `N` parcelas são pagas.

Exemplo:
* `S`: 200
* `N`: 5
* `J`: 1%

Parcelas do financiamento:
`200`; `202`; `204.02`; `206.06`; `208.12`

## Exercício Parte 1 - Escrevendo um Programa

Dado o problema descrito, escreva um programa que calcule as parcelas de um empréstimo para os seguintes valores:
* `S`: 200
* `N`: 5
* `J`: 1%

Esses valores podem ser atribuídos na função `main`, não é necessária leitura pelo teclado. Entretanto, o programa deve ser genérico, ou seja, funcionará para qualquer outro valor atribuído às 3 variáveis.

In [3]:
#include <stdio.h>

int main()
{
    double parcela = 200.00;
    double juros = 1.00;
    int n = 5;
    
    printf("Parcelas do financiamento: %.2lf;", parcela);
    for (int i = 1; i < n; i++) {
        parcela *= 1 + juros/100;
        printf(" %.2lf;", parcela);
    }
}

Parcelas do financiamento: 200.00; 202.00; 204.02; 206.06; 208.12;

## Exercício Parte 2 - Escrevendo um Módulo (Função)

Reescreva o código acima de forma que seu programa faça uso de uma função `proximaParcela` que seja responsável pelo cálculo de cada parcela X do empréstimo.

Utilize as boas práticas de modularização que você aprendeu, evitando dependências do módulo com o programa em que está inserida. Considere que esta função poderia ser eventualmente reusada em outro programa e deveria ser possível fazê-lo sem que ela dependa de nada do programa em que irá se inserir.

A função deve apenas calcular uma única parcela em cada chamada, portanto, ficará a cargo da função principal que chama `proximaParcela` realizar o restante do processo, para que sejam apresentadas todas as parcelas do financiamento.

In [4]:
#include <stdio.h>

double proximaParcela(double parcela, double juros) {
    return parcela * (1 + juros/100);
}

int main()
{
    double parcela = 200.00;
    double juros = 1.00;
    int n = 5;
    
    printf("Parcelas do financiamento: %.2lf;", parcela);
    for (int i = 1; i < n; i++) {
        parcela = proximaParcela(parcela, juros);
        printf(" %.2lf;", parcela);
    }
}

Parcelas do financiamento: 200.00; 202.00; 204.02; 206.06; 208.12;

## Exercício Parte 3 - Minimizando os Parâmetros

Na solução anterior, você deve ter usado vários parâmetros para a comunicação entre o `main` com a função `proximaParcela`, evitando usar variáveis globais. Modifique a forma como você representa o empréstimo, de forma que a função `proximaParcela` receba sempre um único parâmetro capaz de representar todos os valores que estejam relacionados a um empréstimo.

In [9]:
#include <stdio.h>

typedef struct Emprestimo
{
    double parcela;
    double juros;
}Emprestimo;

void proximaParcela(Emprestimo *e){
    e->parcela *= 1 + e->juros / 100;
}

int main()
{
    Emprestimo emprestimo;
    emprestimo.parcela = 200.00;
    emprestimo.juros = 1.00;
    int n = 5;
    
    printf("Parcelas do financiamento: %.2lf;", emprestimo.parcela);
    for (int i = 1; i < n; i++) {
        proximaParcela(&emprestimo);
        printf(" %.2lf;", emprestimo.parcela);
    }
}


Parcelas do financiamento: 200.00; 202.00; 204.02; 206.06; 208.12;

## Exercício Parte 4 - Mantendo o Estado

A função que você codificou na solução anterior provavelmente não é capaz de manter o controle do que chamaremos de **estado do empréstimo**, ou seja, dados de qual a parcela corrente ou da última parcela calculada.

Modifique a função (módulo) `proximaParcela` acima de modo que ela seja capaz de guardar o **estado do empréstimo** por sua conta, sem que o `main` precise ficar informando este estado por parâmetros (por exemplo). Para isso serão permitidas as seguintes modificações:

1. você pode desmembrar a função em mais de uma, por exemplo, uma você chama no começo do empréstimo e outra a cada parcela;

2. você pode usar técnicas de uso de variáveis não recomendadas que geram dependência das funções com o programa em que ela está inserida.

Você deve organizar o código de tal maneira que o `main` informe para as funções os dados do financiamento apenas uma única vez e depois possa solicitar o cálculo da parcela subsequente sem informar tudo novamente. O ideal é que o main não tenha acesso ao modo como as funções vão representar o **estado do empréstimo**, para que toda a lógica do empréstimo fique dentro das funções.

In [18]:
#include <stdio.h>

typedef struct Emprestimo
{
    double parcela;
    double juros;
    int quantidadeDeParcelas;
    int parcelaAtual;
}Emprestimo;

void atualizarParcela(Emprestimo *e){
    e->parcelaAtual++;
}

void proximaParcela(Emprestimo *e){
    e->parcela *= 1 + e->juros / 100;
    atualizarParcela(e);
}

int main()
{
    Emprestimo emprestimo;
    emprestimo.parcela = 200.00;
    emprestimo.juros = 1.00;
    emprestimo.quantidadeDeParcelas = 5;
    emprestimo.parcelaAtual = 1;
    
    printf("Parcelas do financiamento: %.2lf;", emprestimo.parcela);
    while (emprestimo.parcelaAtual < emprestimo.quantidadeDeParcelas) {
        proximaParcela(&emprestimo);
        printf(" %.2lf;", emprestimo.parcela);
    }
}

Parcelas do financiamento: 200.00; 202.00; 204.02; 206.06; 208.12;

## Exercício Parte 5 - Múltiplos Empréstimos

Considere que há múltiplos empréstimos que podem ser controlados em paralelo. A sua função `main` deve ser capaz de apresentar no console as parcelas de mais de um empréstimo de modo paralelo, mantendo um controle para cada **estado de empréstimo** separadamente. Você deve decidir qual das soluções tomará como ponto de partida, se deve modificar a função `main`, as funções de cálculo do empréstimo ou ambas para atender esse requisito da melhor forma possível.

Adote uma solução compacta e generalizável, de tal modo que comporte até 100 empréstimo e, cada novo empréstimo, só exija a informação dos parâmetros de partida, sem expansão do código.

Por exemplo, suponha os seguintes dois empréstimos em paralelo:

### Empréstimo 1
* `S`: 200
* `N`: 5
* `J`: 1%

### Empréstimo 2
* `S`: 500
* `N`: 7
* `J`: 2%

A saída esperada é:
~~~
Emprestimo 1: parcela 1 eh 200.00
Emprestimo 2: parcela 1 eh 500.00
Emprestimo 1: parcela 2 eh 202.00
Emprestimo 2: parcela 2 eh 510.00
Emprestimo 1: parcela 3 eh 204.02
Emprestimo 2: parcela 3 eh 520.20
Emprestimo 1: parcela 4 eh 206.06
Emprestimo 2: parcela 4 eh 530.60
Emprestimo 1: parcela 5 eh 208.12
Emprestimo 2: parcela 5 eh 541.22
Emprestimo 2: parcela 6 eh 552.04
Emprestimo 2: parcela 7 eh 563.08
~~~

O exemplo mostra dois empréstimos, mas a estrutura deve ser genérica o suficiente para controlar N empréstimos em paralelo (até o limite de 100).

O seu programa deve apresentar o exemplo acima como teste.

In [ ]:
#include <stdio.h>

typedef struct Emprestimo
{
    double parcela;
    double juros;
    int quantidadeDeParcelas;
    int parcelaAtual;
}Emprestimo;

void atualizarParcela(Emprestimo *e)
{
    e->parcelaAtual++;
}

void proximaParcela(Emprestimo *e)
{
    e->parcela *= 1 + e->juros / 100;
    atualizarParcela(e);
}

int aindaHaParcela(Emprestimo vetor[100], int qtdeDeEmprestimos){
    for (int i = 0; i < qtdeDeEmprestimos; i++){
        if (vetor[i].parcelaAtual < vetor[i].quantidadeDeParcelas)
            return 1;
    }
    return 0;
}

void imprimirParcelas(Emprestimo vetor[100], int qtdeDeEmprestimos)
{
    while(aindaHaParcela(vetor, qtdeDeEmprestimos)) {
        for (int i = 0; i < qtdeDeEmprestimos; i++){
            if (vetor[i].parcelaAtual < vetor[i].quantidadeDeParcelas){
                printf("Emprestimo %d: parcela %d eh %.2lf\n", i, vetor[i].parcelaAtual, vetor[i].parcela);
                proximaParcela(&vetor[i]);
            }
        }
    }
}

int main()
{
    int qtdeDeEmprestimos = 2;
    Emprestimo emprestimos[100];
    for (int i = 0; i < qtdeDeEmprestimos; i++) {
        scanf(" %lf %d %lf", &emprestimos[i].parcela, &emprestimos[i].quantidadeDeParcelas, &emprestimos.juros);
        emprestimos[i].parcelaAtual = 1;
    }
    
    imprimirParcelas(vetor, qtdeDeEmprestimos);
}